In [2]:
# Import dependencies
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from collections import Counter
import psycopg2
from psycopg2 import Error
from config import DB_PASS

In [3]:
# Connection parameters, yours will be different
param_dic = {
    "host"      : "ec2-34-239-241-121.compute-1.amazonaws.com",
    "database"  : "dfm419pf436p66",
    "user"      : "eulpledavzofae",
    "password"  : DB_PASS
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [4]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    #list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # turn it into a pandas dataframe
    heart_dp = pd.DataFrame(tupples, columns=column_names)
    return heart_dp

In [5]:
# Connect to the database
conn = connect(param_dic)
column_names = ["Pt_ID", "Age", "Sex", "ChestPainType", "RestingBP", "Cholesterol", "FastingBS","RestingEKG","MaxHR","ExerciseAngina","OldPeak","ST_Slope", "HeartDisease"]
# Execute the "SELECT *" query
heart_dp_test = postgresql_to_dataframe(conn, "select * from heart_dp_test", column_names)
heart_dp_test.head()

Connecting to the PostgreSQL database...
Connection successful


,Pt_ID,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingEKG,MaxHR,ExerciseAngina,OldPeak,ST_Slope,HeartDisease
0,10157,43,M,ASY,110,211,0,Normal,161,N,0,Up,Absence
1,10436,45,M,ASY,115,260,0,LVH,185,N,0,Up,Absence
2,10437,64,M,ASY,145,212,0,LVH,132,N,2,flat,Presence
3,10474,52,M,ASY,108,233,1,Normal,147,N,0.1,Up,Absence
4,10477,48,M,NAP,124,255,1,Normal,175,N,0,Up,Absence
